# TWEET STANCE CLASSIFIER

In [1]:
import pandas as pd 
import numpy as np

from models.transfomer_stance_classifier import StanceClassifier

## Data Overview

In [2]:
train_dir = 'data/StanceDataset/train.csv'
test_dir = 'data/StanceDataset/test.csv'

In [3]:
from charset_normalizer import detect
# Detect the encoding
with open(train_dir, "rb") as f:
    detected_encoding = detect(f.read())["encoding"]

In [4]:
train_df = pd.read_csv(train_dir, encoding=detected_encoding, engine='python', on_bad_lines='skip')
test_df = pd.read_csv(test_dir, encoding=detected_encoding, engine='python', on_bad_lines='skip')

In [6]:
train_df.columns

Index(['Tweet', 'Target', 'Stance', 'Opinion Towards', 'Sentiment'], dtype='object')

## Data Preprocess 

In [7]:
train_texts =train_df['Tweet']
train_targets = train_df['Target']
train_stances = train_df['Stance']

## Modelling

In [8]:
classifier = StanceClassifier()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
classifier.train(
    train_texts= train_texts,
    train_targets=train_targets,
    train_stances=train_stances
)